In [77]:
import subprocess as sp
import os
import ffmpeg
import numpy as np
import cv2
from PIL import Image

In [78]:
workdir = os.getcwd()
video_path = workdir + '/test.mp4'
out_filename = workdir + '/frames/out.avi'

probe = ffmpeg.probe(video_path)
video_info = next(x for x in probe['streams'] if x['codec_type'] == 'video')
width = int(video_info['width'])
height = int(video_info['height'])
num_frames = int(video_info['nb_frames'])

In [79]:
# out, err = (ffmpeg
#         .input(video_path)
#         .output(workdir + '/frames/%04d.jpg')
#         .run(capture_stdout=True)
# )

In [80]:
def ffmpeg_reading_process(filename):
    args = (
        ffmpeg
        .input(filename)
        .output('pipe:', crf=23, preset='ultrafast', format='rawvideo', pix_fmt='rgb24')
        .compile()
    )
    return sp.Popen(args, stdout=sp.PIPE)

def ffmpeg_writing_process(filename, width, height):
    args = (
        ffmpeg
        .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height))
        .output(filename, r=str(30), crf=23, preset='ultrafast', movflags='faststart', pix_fmt='rgb24')
        .overwrite_output()
        .compile()
    )
    return sp.Popen(args, stdin=sp.PIPE)

def read_frame(process, width, height):
    frame_size = width * height * 3
    in_bytes = process.stdout.read(frame_size)
    if len(in_bytes) == 0:
        frame = None
    else:
        assert len(in_bytes) == frame_size
        frame = (
            np
            .frombuffer(in_bytes, np.uint8)
            .reshape([height, width, 3])
        )
    return frame

def write_frame(process, frame):
       
    process.stdin.write(
        frame
        .astype(np.uint8)
        .tobytes()
    )

In [81]:
process_read = ffmpeg_reading_process(video_path)
process_write = ffmpeg_writing_process(out_filename, width, height)

while True:
    frame = read_frame(process_read, width, height)
    
    if frame is not None:
        write_frame(process_write, frame)
    
process_read.wait()
process_write.stdin.close()
process_write.wait()

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

KeyboardInterrupt: 

In [ ]:
# process_read = ffmpeg_reading_process(video_path)

# while True:
#     frame = read_frame(process_read, width, height)
#     img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     cv2.imshow("Image", img)
#     k = cv2.waitKey(0)
#     if k == 27:         # wait for ESC key to exit
#         break
    
#     process.stdout.flush()
        
# cv2.destroyAllWindows()
# process.terminate()